<a href="https://colab.research.google.com/github/manmeet3/Deep_Learning2/blob/master/Asg4/multi-task_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook contains the code for a multi-objective movie recommendation model. An example use case for such a model can be seen at http://movielens.org. We follow the multitask example from tensorflow.org

There are two critical parts of multi-task recommenders:
1. They optimize for two or more objectives, and so have two or more losses
2. They share variables between the tasks, allowing for transfer learning

In [3]:
!pip install -q tensorflow-recommenders
!pip install -q --upgrade tensorflow-datasets

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


Tensorflow recommenders is an extension library of core tensorflow.
Ref: https://www.tensorflow.org/resources/libraries-extensions

In [4]:
import os
import pprint
import tempfile
from typing import Dict, Text

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

In [11]:
import tensorflow_recommenders as tfrs

We will use the movielens 100k ratings dataset

In [5]:
ratings = tfds.load('movie_lens/100k-ratings', split="train")
movies = tfds.load('movie_lens/100k-movies', split="train")

# Select basic features
ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"],
    "user_rating": x["user_rating"],
})

movies = movies.map(lambda x: x["movie_title"])

Shuffling and writing examples to /root/tensorflow_datasets/movie_lens/100k-ratings/0.1.0.incomplete8GYDJL/movie_lens-train.tfrecord


Dataset movie_lens downloaded and prepared to /root/tensorflow_datasets/movie_lens/100k-ratings/0.1.0. Subsequent calls will reuse this data.


Shuffling and writing examples to /root/tensorflow_datasets/movie_lens/100k-movies/0.1.0.incomplete8QDIG5/movie_lens-train.tfrecord


Dataset movie_lens downloaded and prepared to /root/tensorflow_datasets/movie_lens/100k-movies/0.1.0. Subsequent calls will reuse this data.


In [7]:
# Randomly shuffle the data and split between train and test
tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

movie_titles = movies.batch(1_000)
user_ids = ratings.batch(1_000_000).map(lambda x: x["user_id"])

unique_movie_titles = np.unique(np.concatenate(list(movie_titles)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

## Multi-task model
The two critical tasks for multi-task recommenders are:
1. Optimization of two or more objectvies -- they must have 2 or more loss functions
2. They share variables between tasks, allowing for transfer learning

In this colab, we define a model which has two tasks:
1. Predict ratings per user
2. Predict movie watches per user

In [29]:
class MovielensModel(tfrs.models.Model):
    def __init__(self, rating_weight: float, retrieval_weight: float) -> None:
        # Taking loss weights in constructor allows multiple model objects with different loss weights
        super().__init__()
        
        self.embedding_dimension=32
        
        # User and movie models
        self.user_model = tf.keras.Sequential([
        tf.keras.layers.experimental.preprocessing.StringLookup(
            vocabulary=unique_user_ids, mask_token=None),
        tf.keras.layers.Embedding(len(unique_user_ids) + 1, self.embedding_dimension)
        ])

        self.movie_model = tf.keras.Sequential([
        tf.keras.layers.experimental.preprocessing.StringLookup(
            vocabulary=unique_movie_titles, mask_token=None),
        tf.keras.layers.Embedding(len(unique_movie_titles) + 1, self.embedding_dimension)
        ])
        
        # A small model to take user and movie embeddings as input and predict ratings
        # This can be made more complicated as long as a scaler is output for prediction
        self.rating_model = tf.keras.Sequential([
            tf.keras.layers.Dense(256, activation="relu"),
            tf.keras.layers.Dense(128, activation="relu"),
            tf.keras.layers.Dense(1),
        ])
        
        # 2 Model Tasks
        # First is the rating task
        self.rating_task: tf.keras.layers.Layer = tfrs.tasks.Ranking(
            loss=tf.keras.losses.MeanSquaredError(),
            metrics=[tf.keras.metrics.RootMeanSquaredError()],
        )
        # Retrieval task for the movies a user may be interested in
        self.retrieval_task: tf.keras.layers.Layer = tfrs.tasks.Retrieval(
            metrics=tfrs.metrics.FactorizedTopK(
                candidates=movies.batch(128).map(self.movie_model)
            )
        )
            
        # The loss weights
        self.rating_weight = rating_weight
        self.retrieval_weight = retrieval_weight
        
    def call(self, features: Dict[Text, tf.Tensor], training=False):
        # Pick out user and movie features and convert them to embeddings
        user_embeddings = self.user_model(features["user_id"])
        movie_embeddings = self.movie_model(features["movie_title"])

        return(
            user_embeddings,
            movie_embeddings,
            # apply multi-layers rating model to concat of user and movie embeddings
            self.rating_model(tf.concat([user_embeddings, movie_embeddings], axis=1)),
        )
        
    def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
        user_embeddings, movie_embeddings, rating_predictions = self(features)

        # We compute the loss for each task
        rating_loss = self.rating_task(
            labels=features["user_rating"],
            predictions=rating_predictions,
        )
        
        retrieval_loss = self.retrieval_task(user_embeddings, movie_embeddings)
        
        # Combine using loss weights
        return (self.rating_weight * rating_loss + self.retrieval_weight * retrieval_loss)

## Rating-specialized model
Depending on the weights provided at initialization time, the MTL model will weight the predictions in favor of that. Here we start with complete emphasis on ratings

In [38]:
model = MovielensModel(rating_weight=1.0, retrieval_weight=0.0)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

In [39]:
cached_train = train.shuffle(100_000).batch(8192).cache()
cached_test = test.batch(4096).cache()

In [40]:
model.fit(cached_train, epochs=10)
metrics = model.evaluate(cached_test, return_dict=True)

print(f"Retrieval top-100 accuracy: {metrics['factorized_top_k/top_100_categorical_accuracy']:.3f}.")
print(f"Ranking RMSE: {metrics['root_mean_squared_error']:.3f}.")

Epoch 1/10


10/10 [==============================] - 4s 416ms/step - root_mean_squared_error: 2.0838 - factorized_top_k/top_1_categorical_accuracy: 2.5000e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0027 - factorized_top_k/top_10_categorical_accuracy: 0.0058 - factorized_top_k/top_50_categorical_accuracy: 0.0292 - factorized_top_k/top_100_categorical_accuracy: 0.0595 - loss: 4.0124 - regularization_loss: 0.0000e+00 - total_loss: 4.0124
Epoch 2/10
10/10 [==============================] - 4s 397ms/step - root_mean_squared_error: 1.1518 - factorized_top_k/top_1_categorical_accuracy: 1.8750e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0026 - factorized_top_k/top_10_categorical_accuracy: 0.0058 - factorized_top_k/top_50_categorical_accuracy: 0.0293 - factorized_top_k/top_100_categorical_accuracy: 0.0595 - loss: 1.3178 - regularization_loss: 0.0000e+00 - total_loss: 1.3178
Epoch 3/10
10/10 [==============================] - 4s 392ms/step - root_mean_squared_error: 1.1159 - factorized_to

## Retrieval-specialized model

In [34]:
model = MovielensModel(rating_weight=0.0, retrieval_weight=1.0)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

In [35]:
model.fit(cached_train, epochs=10)
metrics = model.evaluate(cached_test, return_dict=True)

print(f"Retrieval top-100 accuracy: {metrics['factorized_top_k/top_100_categorical_accuracy']:.3f}.")
print(f"Ranking RMSE: {metrics['root_mean_squared_error']:.3f}.")

Epoch 1/10


10/10 [==============================] - 4s 391ms/step - root_mean_squared_error: 3.6985 - factorized_top_k/top_1_categorical_accuracy: 2.5000e-05 - factorized_top_k/top_5_categorical_accuracy: 0.0014 - factorized_top_k/top_10_categorical_accuracy: 0.0040 - factorized_top_k/top_50_categorical_accuracy: 0.0440 - factorized_top_k/top_100_categorical_accuracy: 0.1054 - loss: 69896.6271 - regularization_loss: 0.0000e+00 - total_loss: 69896.6271
Epoch 2/10
10/10 [==============================] - 4s 385ms/step - root_mean_squared_error: 3.6828 - factorized_top_k/top_1_categorical_accuracy: 0.0011 - factorized_top_k/top_5_categorical_accuracy: 0.0111 - factorized_top_k/top_10_categorical_accuracy: 0.0256 - factorized_top_k/top_50_categorical_accuracy: 0.1392 - factorized_top_k/top_100_categorical_accuracy: 0.2626 - loss: 67519.7464 - regularization_loss: 0.0000e+00 - total_loss: 67519.7464
Epoch 3/10
10/10 [==============================] - 4s 400ms/step - root_mean_squared_error: 3.6746 - f

## Joint Model

In [37]:
model = MovielensModel(rating_weight=1.0, retrieval_weight=1.0)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

In [41]:
model.fit(cached_train, epochs=10)
metrics = model.evaluate(cached_test, return_dict=True)

print(f"Retrieval top-100 accuracy: {metrics['factorized_top_k/top_100_categorical_accuracy']:.3f}.")
print(f"Ranking RMSE: {metrics['root_mean_squared_error']:.3f}.")

Epoch 1/10
10/10 [==============================] - 4s 397ms/step - root_mean_squared_error: 0.9865 - factorized_top_k/top_1_categorical_accuracy: 4.0000e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0036 - factorized_top_k/top_10_categorical_accuracy: 0.0080 - factorized_top_k/top_50_categorical_accuracy: 0.0359 - factorized_top_k/top_100_categorical_accuracy: 0.0690 - loss: 0.9708 - regularization_loss: 0.0000e+00 - total_loss: 0.9708
Epoch 2/10
10/10 [==============================] - 4s 392ms/step - root_mean_squared_error: 0.9771 - factorized_top_k/top_1_categorical_accuracy: 3.6250e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0035 - factorized_top_k/top_10_categorical_accuracy: 0.0077 - factorized_top_k/top_50_categorical_accuracy: 0.0357 - factorized_top_k/top_100_categorical_accuracy: 0.0687 - loss: 0.9517 - regularization_loss: 0.0000e+00 - total_loss: 0.9517
Epoch 3/10
10/10 [==============================] - 4s 405ms/step - root_mean_squared_error: 0.9648 - fa

The last model is a combination of the user-rating prediction and movie recommendation retrieval. In doing so, it is able to retrive movies based on user recommendation

The overall model logic in the notebook is unoptimized for the single or joint models. A more elaborative NN can be implemented in the self.rating_model of the notebook. The rating model takes the embeddings of user and movie models and predicts what a user is likely to rate a recommended movie, were they to watch it.

Reference: https://www.tensorflow.org/recommenders/examples/multitask